In [0]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import copy as cp
from sklearn.model_selection import train_test_split

In [0]:
from google.colab import drive
drive.mount('/gdrive', force_remount=True)

In [0]:
a01=pd.read_csv('/gdrive/My Drive/A01.csv',low_memory=False)
a01_kwd=pd.read_excel('/gdrive/My Drive/A01_KWED_01.xlsx')

In [0]:
a01=a01[a01['ALPHA_GAMMA']==1]
a01['SESS_DT']=pd.to_datetime(a01['SESS_DT'],format='%Y%m%d')
def separate_date(data):
    data["year"] = data["SESS_DT"].dt.year
    data["month"] = data["SESS_DT"].dt.month
    data["day"] = data["SESS_DT"].dt.day
    data["dayofweek"] = data["SESS_DT"].dt.dayofweek
separate_date(a01)
a01['year']=a01['year'].astype('str')
a01['month']=a01['month'].astype('str')
a01['day']=a01['day'].astype('str')
a01['dayofweek']=a01['dayofweek'].astype('str')
a01['hour']=a01['HIT_TM'].apply(lambda x : x.split(':')[0])

In [0]:
a01_kwd=a01_kwd.loc[:,['SECH_KWD','TYPE','BRAND1']]
a01_kwd=a01_kwd.loc[(a01_kwd['TYPE'].notnull())&(a01_kwd['BRAND1'].notnull())]
a01_brand=pd.merge(a01,a01_kwd,how='right',on='SECH_KWD')
a01_brand.shape

(23506, 43)

In [0]:
a01_brand=a01_brand.loc[(a01_brand['SECH_KWD'].notnull())&(a01_brand['ACTION_TYPE']==0)&(a01_brand['USE_TIME']<=3600)]
a01_test=a01_brand.loc[:,['CLNT_ID','SESS_ID','CLNT_GENDER','CLNT_AGE','hour','TRFC_SRC','DVC_CTG_NM','USE_TIME',
                      'ACTION_TYPE','TYPE','BRAND1','SECH_KWD','CATEGORY']]

In [0]:
sech_one=a01_test.groupby('CATEGORY').size().reset_index() #6253
plus=sech_one[sech_one[0]>1] # 5004
plus.columns=['CATEGORY','cate_size']

In [0]:
test1=pd.merge(a01_test,plus,how='left',on='CATEGORY')
test1['CLNT_GENDER']=test1['CLNT_GENDER'].fillna('None')
test1['CLNT_AGE']=test1['CLNT_AGE'].fillna(99)

In [0]:
test2=test1[test1['cate_size'].notnull()]
test2=test2.sort_values('CATEGORY')
test2.reset_index(drop=True,inplace=True)


In [0]:
test_index=test2.drop_duplicates('CATEGORY',keep='last').index

In [0]:
test_index

Int64Index([    1,    12,    43,    78,    84,    86,    95,    98,   109,
              111,
            ...
            17205, 17212, 17215, 17222, 17233, 17241, 17243, 17246, 17248,
            17250],
           dtype='int64', length=2095)

# wide & deep에 label로 사용될 형식 만들기

In [0]:
item_label=test2.groupby(['CATEGORY','BRAND1']).size()
item_label=item_label.unstack()
item_label=item_label.fillna(0)

In [0]:
item_label.reset_index(drop=True,inplace=True)

In [0]:
segment=test2.groupby(['CATEGORY','TYPE']).size()
segment=segment.unstack()
segment=segment.fillna(0)
segment.reset_index(drop=True,inplace=True)

# Doc2Vec

In [0]:
docs=test2.groupby('CATEGORY')['BRAND1'].agg(' '.join).reset_index()

In [0]:
docs2 = [(str(row['BRAND1']).split(), row['CATEGORY']) for idx, row in docs.iterrows()]

In [0]:
from gensim.models import doc2vec
from collections import namedtuple
import multiprocessing

In [0]:
TaggedDocument = namedtuple('TaggedDocument', 'words tags')
tagged_docs = [TaggedDocument(d, [c]) for d, c in docs2]
cores = multiprocessing.cpu_count()

In [0]:
if True:
    doc_vectorizer = doc2vec.Doc2Vec(
    
    dm = 0,            # PV-DBOW
    dbow_words = 0,    # w2v simultaneous with DBOW d2v / default 0
    window = 3,        # distance between the predicted word and context words 
    size = 300,        # vector size 
    alpha = 0.01,     # learning-rate
    min_alpha = 0.01, # min learning-rate
    
    seed = 1,
    
    sample= 1e-5,      # threshold for configuring
    min_count=3,       # ignore with freq lower
    workers=cores,     # multi cpu
    hs = 1             # hierarchical softmax / default 0
    )

    doc_vectorizer.build_vocab(tagged_docs)
    print(str(doc_vectorizer))

    for epoch in range(20):
        doc_vectorizer.train(tagged_docs,epochs=doc_vectorizer.iter,total_examples=doc_vectorizer.corpus_count)
        doc_vectorizer.alpha -= 0.001
        doc_vectorizer.min_alpha = doc_vectorizer.alpha

# To save
   # doc_vectorizer.save('./gdrive/My Drive/Lpoint_Final_Final.model')

/usr/local/lib/python3.6/dist-packages/gensim/models/doc2vec.py:570: UserWarning: The parameter `size` is deprecated, will be removed in 4.0.0, use `vector_size` instead.
  warnings.warn("The parameter `size` is deprecated, will be removed in 4.0.0, use `vector_size` instead.")


Doc2Vec(dbow,d300,n5,hs,mc3,s1e-05,t4)


/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:23: DeprecationWarning: Call to deprecated `iter` (Attribute will be removed in 4.0.0, use self.epochs instead).


In [0]:
item = [doc_vectorizer.infer_vector(doc.words) for doc in tagged_docs]
item_pd = pd.DataFrame(item)
item_pd.head()

,0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20,21,22,23,24,25,26,27,28,29,30,31,32,33,34,35,36,37,38,39,...,260,261,262,263,264,265,266,267,268,269,270,271,272,273,274,275,276,277,278,279,280,281,282,283,284,285,286,287,288,289,290,291,292,293,294,295,296,297,298,299
0,0.000903,0.001354,0.001100,-0.001334,0.000671,-0.001403,0.000583,0.000023,0.001135,0.000535,-0.000714,-0.001204,-0.000905,-0.000941,0.000662,0.000409,0.001237,0.001220,-0.000659,-0.001072,-0.000972,-0.000141,-0.000755,0.001556,-0.001136,0.000600,0.000057,0.000690,-0.001384,-0.001016,-0.000546,-0.000121,0.001106,0.000148,0.001604,-0.001442,-0.001635,-0.000967,-0.000477,0.001108,...,-0.000341,-0.000342,-0.001532,0.001392,-0.000533,-0.001566,0.000226,0.000719,0.000296,-0.000182,0.000926,-0.000538,-0.001446,-0.001245,0.000905,0.001068,0.000407,-0.001454,-0.001528,0.000470,0.000963,-0.001388,0.000936,-0.000860,0.001150,0.001198,-0.001037,0.000017,-0.001389,0.001280,0.001222,0.001071,0.001626,-0.001097,6.556443e-07,0.001122,-0.001448,0.001255,0.000289,0.000281
1,0.000193,0.000966,0.000093,-0.000514,0.001493,0.001499,-0.000582,0.001659,-0.000661,-0.000433,0.000347,0.000538,-0.000396,-0.001176,-0.001029,0.000929,0.000199,0.000587,-0.001452,-0.001228,0.000526,0.000424,0.000940,0.000960,0.000017,-0.000658,-0.001480,-0.000984,0.000308,0.000344,0.000056,-0.000449,-0.000993,-0.000019,0.000623,0.000420,-0.000963,-0.001533,0.000713,-0.000075,...,-0.001439,0.001123,-0.000735,-0.000440,-0.001172,-0.001008,-0.001339,0.001591,-0.001048,-0.000617,-0.000286,-0.001331,0.001344,-0.001290,0.000570,0.001171,-0.000781,0.000749,0.001509,0.000379,-0.001365,0.001192,0.000655,0.000216,0.001133,0.000769,0.001060,0.000691,0.001652,0.000897,0.000678,-0.000460,-0.000546,-0.000102,-1.686035e-04,-0.000740,-0.000064,-0.000801,0.000028,-0.000697
2,0.001325,-0.000452,-0.001197,-0.001163,-0.001389,-0.000227,0.000108,-0.000536,-0.001599,0.001190,0.000561,0.001385,-0.001402,-0.001432,0.000442,0.000403,-0.000708,-0.000879,0.000528,-0.000599,0.001249,0.001222,-0.000208,0.000618,0.000117,-0.001175,-0.000708,-0.000956,0.000321,-0.000622,0.001181,-0.000348,0.000356,-0.000152,0.000929,-0.000231,0.000814,-0.001079,-0.000461,0.000837,...,-0.000628,-0.000029,0.001327,-0.000075,0.001522,0.000271,0.000718,0.001510,0.001619,-0.001157,0.000866,0.001529,0.001123,-0.001243,0.000987,0.000483,0.000136,-0.001534,-0.001117,0.000357,0.000050,0.000070,-0.000287,-0.001261,-0.001119,0.000444,-0.001091,-0.000227,-0.001090,0.000161,0.000740,-0.001009,0.000710,0.000038,-5.863968e-04,0.000599,0.000261,-0.000149,-0.000974,-0.001096
3,0.000883,0.000493,-0.001605,-0.001273,-0.000749,0.001376,-0.001392,0.001138,0.000285,0.001484,0.001117,-0.000669,-0.001276,-0.000939,-0.001219,-0.000780,0.001236,0.000368,-0.000879,-0.001147,0.000239,0.000539,-0.000724,0.000135,-0.000869,0.000979,0.000649,-0.000906,-0.001227,-0.001628,0.000358,0.000403,0.001100,-0.001086,0.000793,-0.000693,0.001156,0.000798,0.000323,0.001129,...,0.000029,0.000608,-0.001192,-0.001371,0.001124,-0.001370,-0.000420,0.001063,-0.000060,-0.000852,-0.000271,-0.000364,0.000451,-0.000963,-0.000763,0.000353,-0.001072,-0.000361,-0.000570,-0.000638,0.000966,-0.000805,0.001521,-0.000134,0.001414,0.001229,0.000933,0.000495,0.000817,-0.000669,0.000143,-0.000307,-0.000040,0.001174,1.195575e-03,0.000623,-0.001085,-0.000059,-0.001501,-0.001027
4,-0.000653,-0.000652,-0.001458,-0.001460,0.001352,-0.001571,-0.001327,0.001803,0.000773,0.000692,0.001406,-0.000918,0.000514,0.001116,-0.001520,0.000834,0.000786,0.001619,0.000805,0.000142,-0.001002,0.000351,0.000269,0.000704,-0.001640,-0.000743,-0.001417,0.000134,0.000612,0.001580,0.001060,0.000169,0.000637,-0.000541,-0.000218,0.001062,-0.001033,-0.000215,-0.000248,-0.000092,...,0.000158,-0.000972,-0.001640,0.001677,-0.001023,-0.001750,-0.000225,0.000885,-0.000131,0.001401,0.000567,-0.000950,-0.000031,-0.000871,-0.000951,-0.000687,-0.000606,0.000362,-0.000209,-0.001592,0.000065,0.000237,0.001170,-0.000908,0.000212,0.000013,-0.001194,0.000

In [0]:
item_pd.shape

(2095, 300)

# wide input 만들기

In [0]:
import tensorflow as tf
from tensorflow.contrib import layers
from datetime import datetime
from sklearn.preprocessing import LabelEncoder
le = LabelEncoder()

In [0]:
len(segment.columns)

346

In [0]:
cross_col =[]
for i in range(345):
    for j in range(345-i):
        cross_col.append([segment.columns[i], segment.columns[i + j + 1]])

In [0]:
len(cross_col)

59685

In [0]:
cross_col2=cross_col[:50000]

In [0]:
item_pd2=cp.deepcopy(item_pd)

segment2=cp.deepcopy(segment)


segment_str = segment2.astype(str)


In [0]:
wide_ = pd.concat([item_pd2, segment_str], axis=1)

In [0]:
wide_

,0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20,21,22,23,24,25,26,27,28,29,30,31,32,33,34,35,36,37,38,39,...,테이블,텐트,토너,토스트기,통조림,티셔츠,틴트,파우치,파운데이션,팔찌,패딩,패팅,팩트,팬티,풋살화,풋크림,핀티,필통,한복,핫도그,핸드워시,핸드크림,핸드폰,핸드폰 케이스,핸드폰거치대,향수,헤어에센스,헤어핀,헬멧,홈웨어,홍삼,화장대,화장품,후드,후드집업,후드티,후라이팬,후리스,휴지,휴지통
0,0.006401,-0.000627,0.000607,-0.002888,0.001390,-0.001761,-0.000381,0.005915,-0.004008,-0.001269,0.006382,0.004229,-0.002473,0.004508,0.002122,0.002198,0.005250,0.002998,0.004308,-0.001268,-0.001290,-0.002548,-0.001412,0.003299,-0.004900,0.001739,-0.005197,-0.002046,0.001141,0.000272,-0.004606,-0.000018,0.003776,-0.002569,0.001907,-0.000890,0.000322,-0.000237,0.000614,0.000329,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,-0.005476,0.011974,-0.002043,-0.004048,0.009738,0.000701,-0.013225,0.016093,0.012632,-0.011551,0.006139,0.012855,-0.008292,-0.002987,-0.002007,0.002425,0.017351,0.003827,0.006385,-0.014555,-0.009592,0.005014,0.001247,0.006018,-0.011536,0.011368,-0.016086,0.002734,0.005801,0.015273,-0.001510,-0.011328,0.001045,-0.013072,0.006163,0.001518,0.003097,-0.007183,0.007546,-0.004286,...,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0
2,-0.002985,0.004792,-0.011082,0.007034,0.000522,-0.003315,-0.004071,0.013017,0.012203,-0.002067,0.003777,-0.007242,-0.002633,0.003732,0.007081,0.002150,0.014762,0.007684,0.002164,-0.000912,-0.008103,-0.002487,0.000579,-0.002164,0.004779,-0.003601,-0.005508,-0.001208,0.003031,0.008044,0.012425,-0.008575,0.008273,0.005320,0.004618,-0.012362,0.010604,0.000417,0.011651,-0.006514,...,0.0,0.0,0.0,0.0,0.0,5.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,-0.039194,-0.008716,-0.025883,-0.008055,0.017743,0.000054,-0.021565,0.036156,0.018950,0.005881,0.033696,0.045350,-0.034369,-0.077285,0.008073,-0.036008,0.053683,0.015132,-0.023098,-0.006223,-0.051842,0.037501,-0.020660,0.004294,0.017333,0.055166,-0.036778,0.024856,0.020397,0.031770,-0.000833,-0.031861,-0.022737,0.001247,0.014032,0.016006,0.031602,-0.038772,0.048656,-0.025128,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,4.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,-0.012478,0.004519,-0.002222,0.010029,-0.004065,0.004767,-0.010234,0.006110,0.004688,-0.002726,0.017371,0.019551,-0.004895,-0.032452,0.007978,-0.008032,0.013854,0.008759,0.004077,-0.000185,-0.011694,0.015515,-0.000890,0.001322,0.003811,0.022851,-0.004146,0.004892,0.016742,-0.004455,-0.008352,-0.000475,-0.010125,-0.003657,0.017632,-0.000654,0.000579,-0.024329,0.013873,0.001123,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,2.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2090,0.001927,0.008409,-0.008267,0.001540,-0.000091,-0.005539,-0.002252,0.003222,0.004063,0.006932,0.003279,-0.000137,-0.001600,-0.003016,0.007135,0.000016,0.008082,0.001282,0.004459,0.006047,-0.002071,-0.001537,0.004764,-0.004557,-0.003265,-0.002321,0.004582,0.001912,-0.003774,0.005841,0.006234,-0.001769,0.000852,0.007240,-0.000400,-0.009911,0.009086,-0.000544,0.008177,-0.006918,...,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,2.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2091,0.000572,0.002803,-0.000415,0.001625,0.004672,0.002008,-0.002161,-0.000463,0.005727,-0.001249,0.000227,-0.000584,-0.003383,-0

In [0]:
def cross_columns(data, cross_colnames):
    for i in cross_colnames:
        tmp = 0
        for j in i:
            tmp += 1
            if tmp == 1:
                cross_data = data[j]
                columns = j
            else:
                cross_data = cross_data + data[j]
                columns = columns + "+" + j
        data[columns] = cross_data
        
    return data

In [0]:
wide_cross = cross_columns(wide_, cross_col2)

In [0]:
wide_data=pd.get_dummies(wide_cross)

In [0]:
wide_data

,0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20,21,22,23,24,25,26,27,28,29,30,31,32,33,34,35,36,37,38,39,...,후드집업_0.0,후드집업_1.0,후드집업_2.0,후드집업_4.0,후드집업_6.0,후드티_0.0,후드티_1.0,후드티_10.0,후드티_2.0,후드티_3.0,후드티_4.0,후드티_5.0,후라이팬_0.0,후라이팬_1.0,후라이팬_2.0,후라이팬_3.0,후라이팬_6.0,후리스_0.0,후리스_1.0,후리스_3.0,후리스_6.0,후리스_7.0,후리스_9.0,휴지_0.0,휴지_1.0,휴지_2.0,휴지_4.0,휴지_5.0,휴지통_0.0,휴지통_1.0,TV+파운데이션_0.00.0,TV+파운데이션_0.01.0,TV+파운데이션_0.02.0,TV+파운데이션_0.03.0,TV+파운데이션_0.05.0,TV+파운데이션_0.08.0,TV+파운데이션_0.09.0,TV+파운데이션_1.00.0,TV+파운데이션_1.02.0,TV+파운데이션_2.00.0
0,0.003145,0.002388,-0.001855,0.001332,0.003396,-0.000613,0.002297,-0.003007,-0.003334,-0.003461,0.001050,-0.001807,0.002842,-0.001600,-0.006040,-0.001841,-0.001073,-0.001840,-0.001542,0.004637,-0.000612,-0.004888,-0.005837,0.004183,0.002175,-0.002633,0.001988,-0.000477,0.002174,0.000419,0.005370,0.001995,0.001001,-0.002649,0.000553,-0.003581,0.003323,0.004056,0.001920,-0.004684,...,1,0,0,0,0,1,0,0,0,0,0,0,1,0,0,0,0,1,0,0,0,0,0,1,0,0,0,0,1,0,1,0,0,0,0,0,0,0,0,0
1,0.001377,-0.002670,0.003717,0.011646,0.014003,0.001671,-0.019045,0.005453,-0.000608,0.000412,0.007336,0.002821,-0.002762,0.001249,-0.009256,0.001355,-0.013379,-0.008239,-0.004782,-0.009342,-0.001176,-0.008080,0.009463,0.010297,-0.003034,-0.007610,-0.007470,-0.003064,0.011576,-0.003990,-0.004711,0.006877,0.005146,-0.004361,-0.009099,-0.002163,0.003657,0.005127,0.008557,-0.002661,...,1,0,0,0,0,0,1,0,0,0,0,0,1,0,0,0,0,1,0,0,0,0,0,1,0,0,0,0,1,0,1,0,0,0,0,0,0,0,0,0
2,0.002002,-0.021015,0.001014,0.000211,0.007020,0.007795,-0.009716,-0.000291,0.001899,-0.005580,0.000743,-0.000536,-0.002264,-0.004046,0.011952,0.006086,-0.013752,0.004264,0.002692,0.007071,0.008554,0.007007,-0.003163,-0.004946,0.001430,0.004218,-0.009155,-0.004867,0.000636,0.002729,0.006307,0.002581,-0.009060,-0.004631,-0.003504,0.000868,0.003294,0.001539,-0.007599,0.000751,...,1,0,0,0,0,1,0,0,0,0,0,0,1,0,0,0,0,1,0,0,0,0,0,1,0,0,0,0,1,0,1,0,0,0,0,0,0,0,0,0
3,-0.010633,-0.002339,0.014851,0.012607,0.037915,-0.004283,-0.012854,0.034748,0.004433,0.014145,0.000144,-0.020992,-0.003020,0.005570,-0.020173,-0.004295,-0.054715,-0.030684,0.007558,-0.006181,0.004089,-0.033755,0.028436,0.011975,-0.046948,0.006550,0.018225,-0.007969,0.039329,-0.023979,-0.035440,0.016750,0.038591,-0.002709,-0.038522,-0.012636,-0.004404,0.042449,0.025966,0.014903,...,1,0,0,0,0,1,0,0,0,0,0,0,1,0,0,0,0,1,0,0,0,0,0,1,0,0,0,0,1,0,1,0,0,0,0,0,0,0,0,0
4,-0.003606,0.002994,0.005161,-0.010581,0.016813,-0.000826,-0.004944,0.017277,0.003741,0.001384,0.007357,-0.013617,-0.001141,0.007699,-0.014868,-0.001342,-0.010326,-0.007260,0.005977,0.008318,-0.007570,-0.018771,0.011495,-0.000898,-0.013303,-0.002014,0.009606,0.006043,0.010550,-0.003972,-0.011337,0.004275,0.017391,-0.007796,-0.013154,-0.002978,-0.002062,0.003512,0.012583,0.002960,...,1,0,0,0,0,1,0,0,0,0,0,0,1,0,0,0,0,1,0,0,0,0,0,1,0,0,0,0,1,0,1,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2090,-0.001747,-0.012160,-0.004466,-0.001203,0.000850,0.004419,-0.004915,-0.001497,-0.001870,0.005289,0.001786,-0.005038,-0.001011,0.002042,0.002598,0.000122,-0.005034,-0.001831,0.003482,0.014580,0.002892,0.004341,-0.002733,-0.005634,-0.001274,0.000964,0.001730,-0.006858,-0.004995,0.003164,0.002000,0.001075,-0.004620,-0.000191,-0.004363,-0.003787,0.001555,0.005044,-0.005067,-0.000672,...,1,0,0,0,0,1,0,0,0,0,0,0,1,0,0,0,0,1,0,0,0,0,0,1,0,0,0,0,1,0,1,0,0,0,0,0,0,0,0,0
2091,-0.003536,0.001418,0.000763,0.007162,0.002574,0.000731,-0.002749,0.002254,0.003797,-0.001310,-0.003255,0.003167,0.003154,-0.000969,-0.001375,0.000770,-0.001439,-0.001278,0.001029,-0.006276,-0.002996,-0.002812,0.002351,0.003211,-0.002601,0.001199,-0.001654,-0.002848,0.002969,-0.004768,0.000484,0.001206,0.002879,0.002178,0.003047,-0.000049,-0.0033

In [0]:
user=test2.groupby(['CATEGORY','CLNT_GENDER','CLNT_AGE']).size().reset_index()
user

,CATEGORY,CLNT_GENDER,CLNT_AGE,0
0,9.0,F,40.0,2
1,14.0,F,40.0,11
2,17.0,F,30.0,31
3,18.0,F,30.0,35
4,19.0,F,30.0,6
...,...,...,...,...
2090,13137.0,F,30.0,8
2091,13143.0,F,30.0,2
2092,13155.0,F,30.0,3
2093,13178.0,F,30.0,2


In [0]:
user_features=user.loc[:,['CLNT_GENDER','CLNT_AGE']]

In [0]:
user_features['CLNT_GENDER']=le.fit_transform(user_features['CLNT_GENDER'])
user_features['CLNT_AGE']=le.fit_transform(user_features['CLNT_AGE'])


user_features.head()

,CLNT_GENDER,CLNT_AGE
0,0,3
1,0,3
2,0,2
3,0,2
4,0,2


In [0]:
deep_data=pd.concat([wide_,user_features],axis=1)
deep_data

,0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20,21,22,23,24,25,26,27,28,29,30,31,32,33,34,35,36,37,38,39,...,토너,토스트기,통조림,티셔츠,틴트,파우치,파운데이션,팔찌,패딩,패팅,팩트,팬티,풋살화,풋크림,핀티,필통,한복,핫도그,핸드워시,핸드크림,핸드폰,핸드폰 케이스,핸드폰거치대,향수,헤어에센스,헤어핀,헬멧,홈웨어,홍삼,화장대,화장품,후드,후드집업,후드티,후라이팬,후리스,휴지,휴지통,CLNT_GENDER,CLNT_AGE
0,0.000903,0.001354,0.001100,-0.001334,0.000671,-0.001403,0.000583,0.000023,0.001135,0.000535,-0.000714,-0.001204,-0.000905,-0.000941,0.000662,0.000409,0.001237,0.001220,-0.000659,-0.001072,-0.000972,-0.000141,-0.000755,0.001556,-0.001136,0.000600,0.000057,0.000690,-0.001384,-0.001016,-0.000546,-0.000121,0.001106,0.000148,0.001604,-0.001442,-0.001635,-0.000967,-0.000477,0.001108,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0,3
1,0.000193,0.000966,0.000093,-0.000514,0.001493,0.001499,-0.000582,0.001659,-0.000661,-0.000433,0.000347,0.000538,-0.000396,-0.001176,-0.001029,0.000929,0.000199,0.000587,-0.001452,-0.001228,0.000526,0.000424,0.000940,0.000960,0.000017,-0.000658,-0.001480,-0.000984,0.000308,0.000344,0.000056,-0.000449,-0.000993,-0.000019,0.000623,0.000420,-0.000963,-0.001533,0.000713,-0.000075,...,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0,3
2,0.001325,-0.000452,-0.001197,-0.001163,-0.001389,-0.000227,0.000108,-0.000536,-0.001599,0.001190,0.000561,0.001385,-0.001402,-0.001432,0.000442,0.000403,-0.000708,-0.000879,0.000528,-0.000599,0.001249,0.001222,-0.000208,0.000618,0.000117,-0.001175,-0.000708,-0.000956,0.000321,-0.000622,0.001181,-0.000348,0.000356,-0.000152,0.000929,-0.000231,0.000814,-0.001079,-0.000461,0.000837,...,0.0,0.0,0.0,5.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0,2
3,0.000883,0.000493,-0.001605,-0.001273,-0.000749,0.001376,-0.001392,0.001138,0.000285,0.001484,0.001117,-0.000669,-0.001276,-0.000939,-0.001219,-0.000780,0.001236,0.000368,-0.000879,-0.001147,0.000239,0.000539,-0.000724,0.000135,-0.000869,0.000979,0.000649,-0.000906,-0.001227,-0.001628,0.000358,0.000403,0.001100,-0.001086,0.000793,-0.000693,0.001156,0.000798,0.000323,0.001129,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,4.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0,2
4,-0.000653,-0.000652,-0.001458,-0.001460,0.001352,-0.001571,-0.001327,0.001803,0.000773,0.000692,0.001406,-0.000918,0.000514,0.001116,-0.001520,0.000834,0.000786,0.001619,0.000805,0.000142,-0.001002,0.000351,0.000269,0.000704,-0.001640,-0.000743,-0.001417,0.000134,0.000612,0.001580,0.001060,0.000169,0.000637,-0.000541,-0.000218,0.001062,-0.001033,-0.000215,-0.000248,-0.000092,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,2.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0,2
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2090,-0.000712,0.000950,-0.000059,0.001664,0.000587,0.001119,0.000825,0.000832,-0.000137,-0.000462,0.000794,0.000813,-0.001016,0.000985,-0.000675,0.000275,0.000420,0.001108,0.000363,-0.000244,-0.000038,-0.001665,-0.000060,-0.000822,-0.000845,-0.000550,-0.000487,0.001217,0.000265,-0.000583,-0.000391,0.000275,0.000257,0.001074,-0.000629,-0.000250,-0.000086,-0.000731,0.000635,-0.000110,...,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,2.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0,2
2091,0.001526,-0.000682,-0.000789,-0.000130,0.000635,0.001376,-0.000703,0.001691,-0.000600,-0.001112,-0.001664,-0.001648,-0.000482,

In [0]:
deep_col = deep_data.columns

In [0]:
wide_data

,0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20,21,22,23,24,25,26,27,28,29,30,31,32,33,34,35,36,37,38,39,...,후드집업_0.0,후드집업_1.0,후드집업_2.0,후드집업_4.0,후드집업_6.0,후드티_0.0,후드티_1.0,후드티_10.0,후드티_2.0,후드티_3.0,후드티_4.0,후드티_5.0,후라이팬_0.0,후라이팬_1.0,후라이팬_2.0,후라이팬_3.0,후라이팬_6.0,후리스_0.0,후리스_1.0,후리스_3.0,후리스_6.0,후리스_7.0,후리스_9.0,휴지_0.0,휴지_1.0,휴지_2.0,휴지_4.0,휴지_5.0,휴지통_0.0,휴지통_1.0,TV+파운데이션_0.00.0,TV+파운데이션_0.01.0,TV+파운데이션_0.02.0,TV+파운데이션_0.03.0,TV+파운데이션_0.05.0,TV+파운데이션_0.08.0,TV+파운데이션_0.09.0,TV+파운데이션_1.00.0,TV+파운데이션_1.02.0,TV+파운데이션_2.00.0
0,0.001188,-0.002927,-0.007358,0.006516,-0.008707,0.003872,-0.004448,0.007946,0.006340,-0.009070,-0.001463,0.001627,0.008463,0.010171,-0.000372,0.004633,0.001390,0.009214,0.002794,0.002611,0.004379,0.003163,0.002079,-0.001606,-0.002241,-0.011368,0.003161,-0.007025,0.012684,0.004400,-0.006659,0.001162,0.002959,-0.011766,0.000905,-0.000141,-0.002996,0.004348,0.004337,-0.002084,...,1,0,0,0,0,1,0,0,0,0,0,0,1,0,0,0,0,1,0,0,0,0,0,1,0,0,0,0,1,0,1,0,0,0,0,0,0,0,0,0
1,-0.012123,0.008017,-0.012451,0.011769,0.014718,0.021470,0.010020,0.010529,0.003890,-0.021305,-0.027190,0.010878,-0.008809,0.006795,-0.011163,0.012285,0.014690,0.003408,-0.000287,0.023396,0.017040,-0.012983,0.027817,-0.000679,0.021368,0.010146,0.014170,0.025968,0.012836,0.000133,-0.009271,-0.010084,0.001225,-0.017145,0.006672,0.001799,-0.013829,0.008141,0.000639,-0.008045,...,1,0,0,0,0,0,1,0,0,0,0,0,1,0,0,0,0,1,0,0,0,0,0,1,0,0,0,0,1,0,1,0,0,0,0,0,0,0,0,0
2,-0.000789,-0.008921,-0.025059,-0.001571,0.030337,0.020546,-0.003737,0.010904,0.007578,-0.015493,0.000645,0.005361,-0.006210,-0.015020,-0.002756,-0.008686,-0.006747,0.024166,-0.005195,0.002569,0.005774,-0.011773,-0.004657,0.004716,-0.010725,0.003925,0.003094,0.012455,-0.011884,-0.010568,-0.004845,-0.007694,0.005435,-0.007214,-0.022391,-0.010851,-0.022485,-0.001746,-0.013583,-0.008778,...,1,0,0,0,0,1,0,0,0,0,0,0,1,0,0,0,0,1,0,0,0,0,0,1,0,0,0,0,1,0,1,0,0,0,0,0,0,0,0,0
3,-0.054587,0.042100,-0.049511,0.004231,0.010998,0.051702,0.019576,-0.001485,-0.031253,-0.064370,-0.138572,0.023188,-0.026315,-0.047590,-0.028966,0.062153,0.037389,-0.027379,0.008455,-0.005696,0.044421,-0.053917,0.035543,0.014893,0.043164,0.035862,0.012087,0.069329,0.024141,-0.042334,0.026820,0.033376,0.011291,-0.009273,0.029236,0.031568,-0.086762,0.035926,-0.026864,0.028966,...,1,0,0,0,0,1,0,0,0,0,0,0,1,0,0,0,0,1,0,0,0,0,0,1,0,0,0,0,1,0,1,0,0,0,0,0,0,0,0,0
4,-0.003275,0.013486,0.006775,-0.000202,-0.007583,-0.002189,-0.016603,0.031816,0.008286,-0.012182,-0.071476,-0.000607,-0.002610,0.010227,-0.005538,0.013027,-0.002621,0.002027,0.025739,0.001109,0.014520,0.002006,0.023113,-0.000964,0.011071,0.004899,0.008767,0.025773,0.005536,-0.030782,0.014520,0.028741,-0.023393,0.007269,0.003682,0.009649,-0.005055,0.023925,-0.016486,0.019733,...,1,0,0,0,0,1,0,0,0,0,0,0,1,0,0,0,0,1,0,0,0,0,0,1,0,0,0,0,1,0,1,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2090,-0.008351,-0.012996,-0.011041,-0.010602,0.017013,0.007884,-0.006260,0.002666,0.000772,-0.008766,0.000011,0.002718,0.008730,-0.006766,0.007463,-0.007486,-0.010275,0.013075,-0.004133,0.000657,0.002265,-0.012018,-0.004917,0.008002,-0.015250,0.008489,0.012699,0.012120,0.001357,0.002764,0.005052,0.001719,0.003662,-0.009504,-0.008593,-0.000533,-0.008165,0.001362,-0.006436,0.003083,...,1,0,0,0,0,1,0,0,0,0,0,0,1,0,0,0,0,1,0,0,0,0,0,1,0,0,0,0,1,0,1,0,0,0,0,0,0,0,0,0
2091,-0.004331,0.005130,-0.007266,0.007946,0.001776,0.011810,0.008515,0.001810,-0.003464,-0.005046,-0.008146,-0.006722,-0.012191,-0.005288,0.004277,-0.002152,0.005576,0.003191,-0.004590,0.001014,0.007289,-0.001958,0.005357,-0.000753,0.002017,0.004110,-0.001068,0.004626,-0.001287,0.002132,0.003366,-0.007786,0.004111,-0.002443,0.013818,0.006963,0.003176,-0.004106,0.

In [0]:
wide_x_train=wide_data.iloc[]
wide_x_test=

In [0]:
wide_x_train,wide_x_test=train_test_split(wide_data,test_size=0.2,random_state=123)

In [0]:
wide_x_train

,0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20,21,22,23,24,25,26,27,28,29,30,31,32,33,34,35,36,37,38,39,...,후드집업_0.0,후드집업_1.0,후드집업_2.0,후드집업_4.0,후드집업_6.0,후드티_0.0,후드티_1.0,후드티_10.0,후드티_2.0,후드티_3.0,후드티_4.0,후드티_5.0,후라이팬_0.0,후라이팬_1.0,후라이팬_2.0,후라이팬_3.0,후라이팬_6.0,후리스_0.0,후리스_1.0,후리스_3.0,후리스_6.0,후리스_7.0,후리스_9.0,휴지_0.0,휴지_1.0,휴지_2.0,휴지_4.0,휴지_5.0,휴지통_0.0,휴지통_1.0,TV+파운데이션_0.00.0,TV+파운데이션_0.01.0,TV+파운데이션_0.02.0,TV+파운데이션_0.03.0,TV+파운데이션_0.05.0,TV+파운데이션_0.08.0,TV+파운데이션_0.09.0,TV+파운데이션_1.00.0,TV+파운데이션_1.02.0,TV+파운데이션_2.00.0
871,0.003329,-0.007181,-0.000954,0.002678,0.002150,0.000959,0.002128,-0.000726,-0.004265,0.003858,0.000013,-0.006880,0.001901,-0.007632,-0.001943,0.001290,-0.009094,-0.000814,0.004291,0.011113,0.005909,-0.002353,-0.001652,-0.005112,-0.002158,0.003291,0.006779,-0.009341,-0.001099,0.001520,0.002111,-0.000908,-0.004451,0.003122,-0.004725,-0.003942,0.002822,0.004706,-0.005051,-0.001341,...,1,0,0,0,0,1,0,0,0,0,0,0,1,0,0,0,0,1,0,0,0,0,0,1,0,0,0,0,1,0,1,0,0,0,0,0,0,0,0,0
120,0.029062,-0.081563,0.056648,0.040851,0.069761,0.002657,-0.089058,0.008745,-0.015982,0.008825,-0.047414,-0.000812,-0.029722,0.002777,0.018363,0.061507,-0.116134,0.030041,0.067567,0.032748,0.019695,-0.030531,0.029655,0.005611,-0.022278,0.024524,-0.031038,-0.056871,0.033824,0.012120,0.022731,0.031671,-0.051676,0.027114,-0.073208,0.000360,0.022539,0.006094,0.021384,0.011961,...,1,0,0,0,0,1,0,0,0,0,0,0,1,0,0,0,0,1,0,0,0,0,0,1,0,0,0,0,1,0,1,0,0,0,0,0,0,0,0,0
1835,-0.000414,0.000221,-0.001562,-0.000492,0.000310,-0.001446,0.000240,0.001047,-0.000436,-0.000228,0.001233,-0.001024,-0.001523,-0.000573,-0.001353,0.000467,0.000960,0.001207,0.000632,0.000180,-0.000914,0.001341,-0.001357,-0.000912,-0.001162,0.000186,0.001103,-0.001287,-0.000819,-0.001315,-0.001424,-0.000367,-0.000999,-0.000649,-0.000702,0.000239,-0.001038,-0.001540,-0.000364,0.001220,...,1,0,0,0,0,1,0,0,0,0,0,0,1,0,0,0,0,1,0,0,0,0,0,1,0,0,0,0,1,0,1,0,0,0,0,0,0,0,0,0
1497,-0.000219,-0.001474,0.002483,0.002690,-0.000843,0.000386,-0.000731,0.000643,0.001614,0.001408,-0.003612,-0.001070,0.000522,-0.001254,0.001084,-0.000021,-0.001000,-0.001988,0.001090,-0.002118,0.000820,-0.003003,0.000494,-0.000744,-0.002700,-0.000515,-0.000259,-0.001216,0.000347,-0.000943,-0.003214,0.002016,0.001507,0.001820,-0.000377,0.002437,0.000011,0.000841,0.000022,0.002800,...,1,0,0,0,0,1,0,0,0,0,0,0,1,0,0,0,0,1,0,0,0,0,0,1,0,0,0,0,1,0,1,0,0,0,0,0,0,0,0,0
31,-0.001865,0.003244,0.002605,0.004282,0.001479,-0.001112,0.000473,-0.000060,0.000956,-0.001515,-0.002987,0.002673,0.002209,0.000863,-0.000717,0.000648,-0.002437,-0.001593,-0.001745,-0.003268,-0.000383,-0.003558,0.002108,0.002784,-0.001696,0.000666,0.001117,-0.002267,0.000303,-0.000977,0.000265,0.001034,0.000448,-0.000272,0.000648,0.002196,-0.001036,0.000847,0.001486,0.001871,...,1,0,0,0,0,1,0,0,0,0,0,0,1,0,0,0,0,1,0,0,0,0,0,1,0,0,0,0,1,0,1,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1593,0.000351,0.001185,-0.000703,0.001173,0.001454,-0.001001,-0.001117,0.000397,0.000948,0.001398,-0.000316,0.001490,0.000737,0.001032,0.000711,-0.001334,0.001169,-0.001495,0.001083,-0.000889,0.000198,-0.000671,-0.001412,-0.000365,-0.000786,-0.001442,0.001461,-0.000398,0.001508,0.001581,0.001428,0.000582,-0.000696,0.000018,0.000701,0.000736,0.001459,-0.001486,-0.000851,-0.000952,...,1,0,0,0,0,1,0,0,0,0,0,0,1,0,0,0,0,1,0,0,0,0,0,1,0,0,0,0,1,0,1,0,0,0,0,0,0,0,0,0
1147,-0.000121,0.001287,0.002637,0.002972,0.002191,-0.000381,-0.001585,0.002394,-0.000207,-0.000023,-0.001171,0.002625,-0.002561,0.004580,-0.002212,0.002915,0.000065,-0.000494,0.003766,-0.001803,0.000239,-0.002062,0.001811,0.003767,-0.001658,0.000319,-0.001769,0.000943,0.000103,-0.003360,-0.000851,0.001357,0.001909,0.002871,-0.000472,-0.000115,0.

In [0]:
wide_x_test

,0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20,21,22,23,24,25,26,27,28,29,30,31,32,33,34,35,36,37,38,39,...,후드집업_0.0,후드집업_1.0,후드집업_2.0,후드집업_4.0,후드집업_6.0,후드티_0.0,후드티_1.0,후드티_10.0,후드티_2.0,후드티_3.0,후드티_4.0,후드티_5.0,후라이팬_0.0,후라이팬_1.0,후라이팬_2.0,후라이팬_3.0,후라이팬_6.0,후리스_0.0,후리스_1.0,후리스_3.0,후리스_6.0,후리스_7.0,후리스_9.0,휴지_0.0,휴지_1.0,휴지_2.0,휴지_4.0,휴지_5.0,휴지통_0.0,휴지통_1.0,TV+파운데이션_0.00.0,TV+파운데이션_0.01.0,TV+파운데이션_0.02.0,TV+파운데이션_0.03.0,TV+파운데이션_0.05.0,TV+파운데이션_0.08.0,TV+파운데이션_0.09.0,TV+파운데이션_1.00.0,TV+파운데이션_1.02.0,TV+파운데이션_2.00.0
1232,-0.002644,0.006286,-0.002100,-0.011939,0.006518,0.003685,-0.003357,0.008113,0.002387,0.001715,0.001235,-0.004930,0.003119,-0.001440,-0.005625,-0.004053,0.002025,-0.002823,-0.000594,0.004790,0.000320,-0.006899,0.009069,-0.000372,-0.005886,0.000425,0.010514,0.005625,0.001988,0.003188,-0.007081,-0.000501,0.001650,-0.007395,0.002742,-0.002276,-0.000127,-0.004178,0.008746,0.006899,...,1,0,0,0,0,1,0,0,0,0,0,0,1,0,0,0,0,1,0,0,0,0,0,1,0,0,0,0,1,0,1,0,0,0,0,0,0,0,0,0
553,0.018436,-0.002282,0.003225,-0.022776,0.038368,0.008286,-0.020177,0.075501,0.016590,-0.018542,0.044065,0.003690,-0.007415,-0.008714,-0.011238,-0.008317,-0.032144,-0.011913,0.009946,0.008834,-0.001336,-0.020596,0.032211,0.000472,-0.013379,0.014479,0.015326,0.009234,0.023232,0.004081,-0.038631,-0.006053,0.038413,-0.028251,-0.012188,-0.007448,0.015289,0.002093,0.021372,0.034760,...,1,0,0,0,0,1,0,0,0,0,0,0,1,0,0,0,0,1,0,0,0,0,0,1,0,0,0,0,1,0,1,0,0,0,0,0,0,0,0,0
1357,-0.001455,-0.059914,-0.007789,-0.008102,0.007271,0.039568,-0.005889,-0.029628,-0.022025,0.022834,0.025971,-0.039005,-0.001573,0.014873,0.018404,-0.003135,-0.047365,0.003115,-0.001102,0.072631,0.047453,0.048345,0.008587,-0.036965,0.009489,0.009889,0.014565,-0.024534,-0.018979,-0.004982,0.012752,0.022534,-0.035939,0.023800,-0.024398,-0.034943,0.033617,0.031455,-0.044670,-0.006379,...,1,0,0,0,0,1,0,0,0,0,0,0,1,0,0,0,0,1,0,0,0,0,0,1,0,0,0,0,1,0,1,0,0,0,0,0,0,0,0,0
630,-0.000023,-0.002876,-0.003142,-0.002183,0.000342,0.001467,0.003797,0.000104,-0.002968,0.001099,0.000739,-0.002016,-0.002297,-0.003383,0.003101,0.000199,0.004694,0.002304,0.000609,0.004437,-0.003088,0.004993,-0.000617,0.001397,0.004922,-0.003526,-0.000057,0.000283,0.001951,0.002008,-0.001277,0.001021,0.000336,0.002205,-0.002961,0.001187,0.001124,-0.002754,0.000565,0.000524,...,1,0,0,0,0,1,0,0,0,0,0,0,1,0,0,0,0,1,0,0,0,0,0,1,0,0,0,0,1,0,1,0,0,0,0,0,0,0,0,0
963,0.014443,-0.008154,0.014233,-0.012803,0.023695,0.001069,0.001632,-0.001675,-0.018002,0.004030,0.020389,-0.022266,-0.011483,-0.019591,-0.017294,-0.004054,-0.039881,-0.007584,-0.001683,0.032723,0.006921,-0.010928,0.016711,-0.012699,-0.000448,0.001773,0.038489,-0.023778,-0.004919,0.002001,0.001076,0.006212,-0.005690,0.003286,-0.008781,-0.006568,0.017651,0.007141,-0.010476,-0.000701,...,0,1,0,0,0,0,1,0,0,0,0,0,1,0,0,0,0,0,0,0,1,0,0,1,0,0,0,0,1,0,1,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1956,0.018704,0.008644,-0.005700,-0.001724,0.005587,-0.000732,0.000957,-0.017689,-0.006209,-0.008614,0.007654,-0.002370,-0.000222,-0.013194,-0.016744,-0.003126,-0.023578,-0.008994,0.002207,0.007117,0.000641,-0.004057,-0.012797,0.010474,0.003952,-0.007695,0.002823,-0.020484,0.005825,0.006590,0.018475,0.006759,0.003891,-0.009557,0.015132,-0.000820,0.011279,0.018145,0.004049,-0.005189,...,1,0,0,0,0,1,0,0,0,0,0,0,1,0,0,0,0,1,0,0,0,0,0,1,0,0,0,0,1,0,1,0,0,0,0,0,0,0,0,0
381,0.006304,0.002211,0.002936,-0.016833,0.001752,0.003255,-0.004402,0.006637,0.003710,-0.004260,0.005736,-0.002187,0.001055,0.000977,-0.009835,-0.000538,-0.005245,-0.000495,0.001996,0.003491,-0.004799,-0.001995,0.004089,-0.001108,-0.000824,0.000988,0.005087,0.004246,-0.002329,0.000174,-0.009724,-0.005690,0.009185,-0.004077,0.001405,0.001166,0

In [0]:
deep_x_train=deep_data.iloc[wide_x_train.index,:]
deep_x_test=deep_data.iloc[wide_x_test.index,:]

In [0]:
train_label=item_label.iloc[wide_x_train.index,:]

In [0]:
test_label=item_label.iloc[wide_x_test.index,:]

In [0]:
tf.reset_default_graph()

# Wide & Deep learning 학습 모델

In [0]:
# parameters
# keep_prob = 0.5
learning_rate = 1e-4
training_epochs = 10
batch_size = 256

wide_size = len(wide_x_train.columns)
deep_size = len(deep_x_train.columns) 
deep_embed_size = 2 # 임베딩할 컬럼 개수

output_size = len(train_label.columns)

He = tf.contrib.layers.variance_scaling_initializer()

# input place holders
tf.reset_default_graph()
Wide = tf.placeholder(tf.float32, [None, wide_size])
Deep_un_embed = tf.placeholder(tf.float32, [None, deep_size - deep_embed_size])
Deep_embed = tf.placeholder(tf.int32, [None, deep_embed_size])

Y = tf.placeholder(tf.float32, [None, output_size])
train_mode = tf.placeholder(tf.bool, name='train_mode')

The TensorFlow contrib module will not be included in TensorFlow 2.0.
For more information, please see:
  * https://github.com/tensorflow/community/blob/master/rfcs/20180907-contrib-sunset.md
  * https://github.com/tensorflow/addons
  * https://github.com/tensorflow/io (for I/O related ops)
If you depend on functionality not listed there, please file an issue.



In [0]:
no_class_max = 42
embedding_size = 5

deep_embed_var = tf.Variable(tf.random_uniform([no_class_max, embedding_size], -1.0, 1.0))
deep_embed = tf.nn.embedding_lookup(deep_embed_var, Deep_embed)
deep_embed_re = tf.reshape(deep_embed, [-1, embedding_size * deep_embed_size])

In [0]:
Deep = tf.concat([Deep_un_embed, deep_embed_re], axis=1)

In [0]:
Deep 

<tf.Tensor 'concat:0' shape=(?, 656) dtype=float32>

In [0]:
A = tf.get_variable("A", shape=[wide_size, output_size])
b = tf.Variable(tf.random_normal([output_size]))

wide = tf.add(tf.matmul(Wide, A), b)

In [0]:
W1 = tf.get_variable("W1", shape=[656, 512], initializer=He)
b1 = tf.Variable(tf.random_normal([512]))
L1 = tf.nn.relu(tf.matmul(Deep, W1) + b1)

W2 = tf.get_variable("W2", shape=[512, 256], initializer=He)
b2 = tf.Variable(tf.random_normal([256]))
L2 = tf.nn.relu(tf.matmul(L1, W2) + b2)

W3 = tf.get_variable("W3", shape=[256, 128], initializer=He)
b3 = tf.Variable(tf.random_normal([128]))
L3 = tf.nn.relu(tf.matmul(L2, W3) + b3)

W4 = tf.get_variable("W4", shape=[128, output_size], initializer=He)
b4 = tf.Variable(tf.random_normal([output_size]))

deep = tf.add(tf.matmul(L3, W4), b4)

In [0]:
hypothesis = wide + deep

In [0]:
global_step = tf.Variable(initial_value=0, name='global_step', trainable=False)

In [0]:
cost = tf.reduce_mean(tf.nn.sigmoid_cross_entropy_with_logits(logits=hypothesis, labels=Y))
optimizer = tf.train.AdamOptimizer(learning_rate=learning_rate).minimize(cost, global_step=global_step)

Instructions for updating:
Use tf.where in 2.0, which has the same broadcast rule as np.where


In [0]:
config = tf.ConfigProto()
config.gpu_options.allow_growth=True
sess = tf.Session(config=config)
sess.run(tf.global_variables_initializer())

In [0]:

def Model_train(training_epochs = 30):
    for epoch in range(training_epochs):
        avg_cost = 0
        total_batch = int(len(deep_x_train) / batch_size)

        for i in range(0, len(deep_x_train), batch_size):
            batch_wide = wide_x_train[i:i+batch_size]
            batch_deep = deep_x_train[i:i+batch_size][deep_col[:-2]]
            batch_deep_embed = deep_x_train[i:i+batch_size][deep_col[-2:]]
            
            batch_label = train_label[i:i+batch_size]

            feed_dict_train = {Wide: batch_wide,
                               Deep_un_embed: batch_deep,
                               Deep_embed: batch_deep_embed,
                               Y: batch_label}
            i_global, opt, c = sess.run([global_step, optimizer, cost],
                                        feed_dict=feed_dict_train)

            avg_cost += c/total_batch

        if (epoch % 1 == 0) or (epoch == training_epochs - 1):
            time = datetime.now().strftime('%Y-%m-%d %H:%M:%S')
            print("[epoch: {:>4}][Cost: {:>.5f}][{}]".format(epoch, avg_cost, time))

            
    print('Learning Finished!')

In [0]:
sess.run(tf.global_variables_initializer())
Model_train(10)

[epoch:    0][Cost: 0.31319][2020-02-09 02:43:59]
[epoch:    1][Cost: 0.16205][2020-02-09 02:44:16]
[epoch:    2][Cost: 0.15055][2020-02-09 02:44:33]
[epoch:    3][Cost: 0.12880][2020-02-09 02:44:51]
[epoch:    4][Cost: 0.10674][2020-02-09 02:45:08]
[epoch:    5][Cost: 0.08712][2020-02-09 02:45:25]
[epoch:    6][Cost: 0.07019][2020-02-09 02:45:42]
[epoch:    7][Cost: 0.05514][2020-02-09 02:45:59]
[epoch:    8][Cost: 0.04261][2020-02-09 02:46:16]
[epoch:    9][Cost: 0.03134][2020-02-09 02:46:33]
Learning Finished!


In [0]:
wide_x_test

,0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20,21,22,23,24,25,26,27,28,29,30,31,32,33,34,35,36,37,38,39,...,등산화+여성수영복_2.00.0,등산화+여성수영복_8.00.0,등산화+여성화_0.00.0,등산화+여성화_0.01.0,등산화+여성화_0.02.0,등산화+여성화_0.03.0,등산화+여성화_1.00.0,등산화+여성화_2.00.0,등산화+여성화_8.00.0,등산화+염색약_0.00.0,등산화+염색약_0.02.0,등산화+염색약_0.03.0,등산화+염색약_1.00.0,등산화+염색약_2.00.0,등산화+염색약_8.00.0,등산화+영양제_0.00.0,등산화+영양제_0.01.0,등산화+영양제_0.010.0,등산화+영양제_0.02.0,등산화+영양제_0.026.0,등산화+영양제_0.03.0,등산화+영양제_0.05.0,등산화+영양제_0.06.0,등산화+영양제_0.08.0,등산화+영양제_1.00.0,등산화+영양제_2.00.0,등산화+영양제_8.00.0,등산화+오일_0.00.0,등산화+오일_0.01.0,등산화+오일_0.02.0,등산화+오일_0.07.0,등산화+오일_1.00.0,등산화+오일_2.00.0,등산화+오일_8.00.0,등산화+오픈숄더_0.00.0,등산화+오픈숄더_0.01.0,등산화+오픈숄더_0.04.0,등산화+오픈숄더_1.00.0,등산화+오픈숄더_2.00.0,등산화+오픈숄더_8.00.0
1232,-0.000840,0.000444,-0.000950,0.000683,0.000067,0.001603,-0.000245,-0.001053,0.000110,0.000363,0.002472,0.003036,-0.001067,-0.008059,0.001261,-0.001852,0.000834,-0.000220,-0.001166,0.000115,-0.001919,0.000409,-0.002285,-0.000176,-0.000016,0.003242,-0.000924,0.001519,0.002489,-0.004229,-0.001319,0.001990,-0.004612,-0.001192,0.004246,0.001308,0.002228,-0.004016,0.000416,-0.000283,...,0,0,1,0,0,0,0,0,0,1,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,1,0,0,0,0,0
553,-0.004976,-0.002999,0.003278,0.007902,-0.007243,0.002792,-0.011865,0.020371,0.018633,-0.010369,0.029154,0.018788,-0.002209,-0.037149,0.025992,-0.010566,0.020323,0.005383,0.011154,-0.008558,-0.023156,0.016692,-0.005365,-0.000572,0.003578,0.029557,-0.006610,0.000375,0.028876,-0.010478,-0.007589,0.003035,-0.017776,-0.009839,0.034200,-0.008098,0.014839,-0.017386,0.025068,0.009472,...,0,0,1,0,0,0,0,0,0,1,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,1,0,0,0,0,0
1357,-0.002619,0.022978,-0.029722,0.007911,0.004338,-0.021645,-0.009626,0.006827,0.013647,0.014639,0.006441,0.011353,-0.009040,-0.003561,0.015478,0.008741,0.027183,0.027036,0.006874,0.013475,-0.005198,-0.011421,0.013277,-0.016270,-0.014113,-0.001629,-0.006272,0.004562,-0.008571,0.021376,0.025941,-0.010974,-0.003635,0.014929,-0.008319,-0.022638,0.025779,-0.005807,0.026941,-0.036502,...,0,0,1,0,0,0,0,0,0,1,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,1,0,0,0,0,0
630,-0.001345,0.000625,-0.000091,0.000352,-0.001446,0.001285,-0.000466,-0.000271,0.000645,-0.000157,0.000810,0.000891,-0.000752,-0.001415,-0.001157,0.000899,-0.001049,0.000938,0.000437,0.001286,0.000945,-0.000144,0.000362,0.001641,-0.000485,-0.000918,0.000750,-0.000045,0.000973,0.001574,0.000823,-0.000074,-0.000639,0.000289,0.000252,-0.001430,0.000934,-0.001138,-0.000952,0.001103,...,0,0,1,0,0,0,0,0,0,1,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,1,0,0,0,0,0
963,-0.000628,0.000547,-0.006542,0.003062,0.007954,-0.005264,-0.004583,0.013547,0.002441,0.004163,0.021983,0.006070,-0.010825,-0.017252,0.010141,-0.002837,0.025192,0.010790,0.010464,-0.002962,-0.012105,-0.002690,-0.004847,-0.005403,-0.004515,0.008472,-0.008113,0.002849,0.002079,0.010898,0.004559,0.000338,0.005577,0.010076,0.012898,-0.014459,0.010753,-0.004917,0.015284,-0.004595,...,0,0,1,0,0,0,0,0,0,1,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,1,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
382,-0.003722,0.006453,0.000961,0.000080,0.004061,-0.000170,-0.001024,-0.001512,-0.000437,-0.002380,0.000932,0.004931,-0.002707,0.002136,-0.000225,0.001850,-0.001244,0.005225,0.002003,-0.000255,-0.001499,0.000445,-0.000486,0.003284,-0.004312,0.001357,-0.002487,-0.000703,-0.000781,0.000454,-0.004554,-0.001227,-0.000377,-0.004341,0.000620,0.002776,-0.000472,-0.002193,0.001677,0.001229,...,0,0,1,0,0,0,0,0,0,1,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,1,0,0,0,0,0
1048,-0.003283,0.002546,0.002420,-0.001151,-0.004156,0.002256,-0.003737,0.005113,0.002106,-0.001241,0.004421,-0.002025,-0.001015,-0.004641,0.00348

In [0]:
YY = tf.equal(Y , 1)
XX = tf.greater(tf.sigmoid(hypothesis), 0.5)
correct = tf.equal(XX, YY)
#score = sess.run(correct, feed_dict={Wide: wide_x_test, Deep: deep_x_test[deep_col[:-2]], Y: test_label})
accuracy = tf.reduce_mean(tf.cast(correct, 'float'))

feed_dict_test = {Wide: wide_x_test,
                  Deep_un_embed: deep_x_test[deep_col[:-2]],
                  Deep_embed: deep_x_test[deep_col[-2:]],
                  Y: test_label}

print("Accuracy: {:>.5f}".format(sess.run(accuracy, feed_dict=feed_dict_test)))

Accuracy: 0.99671


## 추천 모델의 정확도는 다른 방식으로 계산되서 그에 맞게 계산했습니다.

In [0]:
test_label

BRAND1,1159스튜디오,LG전자,SK매직,TS,가누다,가락시장ⓔ몰,가버,가보흙침대,갤럭시,갭,게스,게이즈샵,겐조,겐코,겔랑,고려은단,고세,골든구스,골든듀,골든벨,구찌,굽네몰,그레고리,그룬트,그린핑거,까르뜨블랑슈남성,까스텔바작,꼬떼따블,꼬무신,꼼빠니아,나라데코,나비타월드,나스,나이기,나이스클랍,나이키,나이키/엔비에이,나이키골프,나이키수영복,나인,...,핑골프,하기스,하림,한국도자기,한국생활건강,한국조폐공사,한만두식품,한샘,한스타일슈,한율,한일카페트,핫핑,해피콜,햇반,행텐,허닭,허스키,허츠앤베이,헌트키즈,헤라,헤르본,헤이즈,헤지스,헤지스ACC,헤지스골프,헤지스레이디스,헤지스키즈,헤지스홈,헬씨휴먼,헹켈,홀리넘버세븐,홈스웰,홈앤하우스,홍이장군,홍진경의 더김치,홍진경의 더만두,후라밍고,휘슬러,휠라,휠라키즈
1232,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
553,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1357,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
630,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
963,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
382,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1048,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
903,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
30,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0


In [0]:
test_label.T[1232].idxmax()

'지오다노'

In [0]:
wide_x_test

,0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20,21,22,23,24,25,26,27,28,29,30,31,32,33,34,35,36,37,38,39,...,물+탈취제_1.00.0,물+탈취제_2.00.0,물+탈취제_3.00.0,물+탈취제_4.00.0,물+탈취제_5.00.0,물+탈취제_7.00.0,물+턱시도_0.00.0,물+턱시도_0.025.0,물+턱시도_1.00.0,물+턱시도_2.00.0,물+턱시도_3.00.0,물+턱시도_4.00.0,물+턱시도_5.00.0,물+턱시도_7.00.0,물+텀블러_0.00.0,물+텀블러_0.06.0,물+텀블러_1.00.0,물+텀블러_2.00.0,물+텀블러_3.00.0,물+텀블러_4.00.0,물+텀블러_5.00.0,물+텀블러_7.00.0,물+테이블_0.00.0,물+테이블_0.01.0,물+테이블_0.02.0,물+테이블_0.03.0,물+테이블_1.00.0,물+테이블_2.00.0,물+테이블_3.00.0,물+테이블_4.00.0,물+테이블_5.00.0,물+테이블_7.00.0,물+텐트_0.00.0,물+텐트_0.011.0,물+텐트_1.00.0,물+텐트_2.00.0,물+텐트_3.00.0,물+텐트_4.00.0,물+텐트_5.00.0,물+텐트_7.00.0
1232,-0.002223,0.002785,-0.000716,-0.004042,0.003121,0.000504,-0.001415,0.003639,0.001062,0.000945,0.000280,-0.003953,0.000932,0.000543,-0.002058,-0.000455,-0.000145,-0.000838,0.000253,0.001597,-0.000353,-0.003023,0.003915,-0.000973,-0.003378,0.000889,0.004984,0.003548,0.002384,0.000501,-0.003347,0.000109,0.002120,-0.002242,0.000039,-0.001809,-0.000020,-0.001671,0.003959,0.002665,...,0,0,0,0,0,0,1,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0
553,-0.003190,0.006193,0.003767,-0.000483,0.022967,-0.002411,-0.018595,0.033327,0.006054,-0.002714,0.015603,-0.007255,-0.006763,0.006542,-0.011349,0.005828,-0.028758,-0.008171,0.003258,-0.003024,-0.003803,-0.010761,0.016987,0.002303,-0.013635,0.006463,0.010515,0.011829,0.023596,-0.005270,-0.019455,0.003199,0.025573,-0.010211,-0.010395,-0.009388,0.005465,0.000089,0.016516,0.015510,...,0,0,0,0,0,0,1,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0
1357,0.008278,-0.042850,-0.017893,-0.012999,0.004404,0.020847,-0.005846,-0.012770,-0.008142,0.008424,0.016461,-0.018277,-0.001320,0.010732,0.016235,0.004961,-0.021021,0.012338,0.003748,0.057120,0.017250,0.016808,-0.000918,-0.016993,-0.002126,-0.004246,-0.005197,-0.020667,-0.018816,0.006468,0.006445,0.006516,-0.023958,0.004182,-0.016765,-0.012821,0.024809,0.019372,-0.017069,-0.015021,...,0,0,0,0,0,0,1,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0
630,-0.000008,-0.000959,-0.001047,-0.000728,0.000114,0.000489,0.001266,0.000035,-0.000989,0.000366,0.000246,-0.000672,-0.000766,-0.001128,0.001034,0.000066,0.001565,0.000768,0.000203,0.001479,-0.001029,0.001664,-0.000206,0.000466,0.001641,-0.001175,-0.000019,0.000094,0.000650,0.000669,-0.000426,0.000340,0.000112,0.000735,-0.000987,0.000396,0.000375,-0.000918,0.000188,0.000175,...,0,0,0,0,0,0,1,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0
963,0.005174,-0.006027,0.002779,-0.003552,0.011462,0.000031,-0.002004,0.001327,-0.005231,0.004498,0.008216,-0.014115,-0.003118,-0.006301,-0.005879,0.001923,-0.022722,-0.000179,0.002033,0.022716,0.003576,-0.007593,0.003464,-0.006671,-0.005411,0.003484,0.013882,-0.008655,0.000789,0.003165,-0.003812,0.002622,-0.002720,0.000903,-0.011601,-0.003142,0.009841,0.006951,-0.000160,-0.001657,...,0,0,0,0,0,0,1,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
382,-0.001064,0.002975,-0.002984,0.000011,-0.000279,-0.001633,-0.003275,-0.001157,0.001033,-0.001550,0.001291,0.004704,-0.000158,-0.002382,-0.006149,-0.000048,-0.002822,-0.000401,-0.006725,-0.000758,-0.000868,-0.002899,0.002244,0.002232,0.001378,-0.002290,-0.001968,-0.001277,0.000909,-0.001378,-0.000578,-0.003211,0.002520,-0.000004,0.000061,-0.001192,0.003374,0.001789,-0.000276,-0.000533,...,0,0,0,0,0,0,1,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0
1048,-0.000637,-0.001068,-0.002067,-0.000678,0.006248,-0.003346,-0.002493,0.008548,0.001408,0.000501,0.001383,-0.000857,-0.001018,0.003602,-0.002317,0.002174,-0.000631,-0.001915,0.001422,-0.005306,-0.001365,-0.003240,0.002879,-0.0017

In [0]:
def get_acc(score_matrix, top_n, test_matix):
    avg_acc = 0
    for i in range(len(score_matrix)):
        top = score_matrix.iloc[i].nlargest(top_n).index
        tmp = 0
        for j in range(len(top)):
            if top[j] in test_matix["item"][i].split():
                tmp += 1

        acc = tmp / len(top)
        avg_acc += acc / len(score_matrix)

    return avg_acc

In [0]:
def Model_eval(training_epochs = 10):
    for epoch in range(training_epochs):
        avg_cost = 0
        total_batch = int(len(deep_x_train) / batch_size)

        for i in range(0, len(deep_x_train), batch_size):
            batch_wide = wide_x_train[i:i+batch_size]
            batch_deep = deep_x_train[i:i+batch_size][deep_col[:-2]]
            batch_deep_embed = deep_x_train[i:i+batch_size][deep_col[-2:]]
            
            batch_label = train_label[i:i+batch_size]

            feed_dict_train = {Wide: batch_wide,
                               Deep_un_embed: batch_deep,
                               Deep_embed: batch_deep_embed,
                               Y: batch_label}
            
            feed_dict_test = {Wide: wide_x_test,
                              Deep_un_embed: deep_x_test[deep_col[:-2]],
                              Deep_embed: deep_x_test[deep_col[-2:]],
                              Y: test_label}
            
            i_global, opt, c = sess.run([global_step, optimizer, cost], feed_dict=feed_dict_train)

            avg_cost += c/total_batch
            
        score = sess.run(tf.sigmoid(hypothesis), feed_dict=feed_dict_test)
        score_pd = pd.DataFrame(score, columns=train_label.columns)
      #  acc = get_acc(score_pd, 10, test_label)

        if (epoch % 1 == 0) or (epoch == training_epochs - 1):
            time = datetime.now().strftime('%Y-%m-%d %H:%M:%S')
            main = "[Epoch: {:>4}][Cost: {:>.5f}][Hit rate of Top 10: {:>.5f}]"
            #print(main.format(epoch, avg_cost, acc))             
            #print("[global_step: {:>4}][{}]".format(i_global, time))

  
            
    print('Learning Finished!')

In [0]:
sess.run(tf.global_variables_initializer())
Model_eval(10)

Learning Finished!


In [0]:
score = sess.run(tf.sigmoid(hypothesis), feed_dict=feed_dict_test)

In [0]:
score = pd.DataFrame(score, columns=train_label.columns)

In [0]:
s=test_label.T.iloc[:,197].reset_index()
s.sort_values(429)

,BRAND1,429
0,1159스튜디오,0.0
469,올리브데올리브,0.0
470,올젠,0.0
471,옷자락,0.0
472,와이드앵글,0.0
...,...,...
709,휠라키즈,0.0
414,에스케이투,1.0
427,에이치씨,1.0
416,에스티로더,1.0


In [0]:
que=score.T[197].reset_index()
que.sort_values(197,ascending=False).head(20)

,BRAND1,197
355,써스데이아일랜드,0.268546
361,아디다스,0.197628
35,나이키,0.175431
501,입생로랑,0.145966
181,맥,0.129955
303,설화수,0.099586
447,엠엘비,0.089252
235,베베드피노,0.087306
584,크록스,0.086284
416,에스티로더,0.083075
